## Here I try to reproduce the results from the professor, even if he has made some mistakes.

In [2]:
import math
import numpy as np
import numpy.linalg as la
import pandas as pd
import matplotlib.pyplot as plt

data_file = pd.ExcelFile("NYU_Spring_2019_Assignment_3_data.xls")
#read first sheet
bond_index = pd.read_excel(data_file, "VBTLX", skiprows=[0])
columns = bond_index.columns.values
columns[0] = "Year"
#read second sheet
int_growth = pd.read_excel(data_file, "VWILX", skiprows=[0])
columns = int_growth.columns.values
columns[0] = "Year"
#read third sheet
metal = pd.read_excel(data_file, "VGPMX", skiprows=[0])
columns = metal.columns.values
columns[0] = "Year"
#read fourth sheet
stock_index = pd.read_excel(data_file, "VTSAX", skiprows=[0])
columns = stock_index.columns.values
columns[0] = "Year"
#read fifth sheet
int_value = pd.read_excel(data_file, "VTRIX", skiprows=[0])
columns = int_value.columns.values
columns[0] = "Year"
#read sixth sheet
reit = pd.read_excel(data_file, "VGSLX", skiprows=[0])
columns = reit.columns.values
columns[0] = "Year"
#read sixth sheet
money_market = pd.read_excel(data_file, "VMRXX", skiprows=[0])
columns = money_market.columns.values
columns[0] = "Year"

## Some emails between me and professors

### With Professor Brown
Hi Professor Brown,

Recall that in assignment 3, we were given quarterly returns of six assets and are asked to solve for the sharpe ratios. In that assignment, we need to calculate the covariance matrix from the return data. However, I tried various methods, including average the quarterly returns, or compounding, but none of them yield me the same results as your answers. I have come to the TA's office hours and we still cannot figure it out how you get the matrix. Since this matrix is critical, as otherwise we cannot know whether the later on calculations we did are correct or not, I was wondering could you please share the spreadsheet or code, when you compute the results with us, so that we can see how to do that?
_____________________________

What are the differences? I may have forgotten to divide my covariance matrix by the number of observations, are your numbers off by that factor?
Did you start by subtracting off the risk-free rate from all price series, and then subtract the mean from each one? Once you do that, the covariance matrix is just the return matix transpose times the return matrix, then divided element by element by the number of observations.
I can send you a spreadsheet with the calculation if that's not clear.

_____________________________

Hi Professor Brown,

Thanks for your reply. No it is not off by a factor of the number of observations. Let me mention that "subtracting off the risk-free rate from all price series" is not required in the calculation of covariance, but should not affect the result, as Var(X) = Var(X+c). However, following your instructions, I still cannot get your results, and the difference is quite large. Could you please send me a spreadsheet showing how you have done the calculation? Say just the variance of stock index return (your answer is 0.0130), would be sufficient for me to grasp the process.

_______________________________

Subtracting off the risk-free return certainly is necessary if you want to get the covariance matrix of excess returns. The risk-free rate is not a constant, despite the name, it's the money market rate.



### With Dr. Robert V. Kohn

Hi Professor Kohn,

My name is Zeshun and I took your math finance course three semesters ago. I was troubled by an optimization problem and I hope I can get some help. Recall that in class we discussed the CAPM model and how to find the market portfolio by maximizing the slope, i.e. 
$$\max \frac{w \cdot  m - R_f}{\sqrt{w^T C w}},$$
where w is the weight vector, m the mean returns of n assets, C the var-cov matrix. In class we use method of lagrangian multiplier to maximize this, subject to weights sum up to 1. 
Now, I am trying to apply this to real data, and the method of lagrangian multiplier is inconvenient. Then, I was told that the above maximizing problem is equivalent to 
$$ \min w^T Cw - k (w \cdot m - R_f),$$ with k being a scale we can use to normalize the w vector. I am confused how these to problems are equivalent? It doesn't seem so as the second one is minimizing risk subject to some specified level of return.

_______________________________

Hi Zeshun,

I agree with you that the two ideas are not equivalent if C is positive 
definite. In particular, minimizing risk for given mean return is
equivalent to the second equation you wrote (with an undetermined k), by 
the method of Lagrange multipliers.

But: we can treat the risk-free asset as being "just" another asset. This 
makes the covariance matrix (n+1)x(n+1) with a null-direction associated 
with the risk-free asset. We know that the optimal tradeoff of risk and 
return is associated with mixtures of the "market portfolio" and the 
risk-free asset, and the points achieved in the (risk,return) plane are on 
a half-line going through (0,R_f) and (\sigma_M,\mu_M). So if you minimize 
risk for fixed return (greater than R_f) in this setting, you're assured 
of picking up a point on this half-line.

# Question 1, Calculate excess return
This is done by element wise substraction of risk-free rate.

In [3]:
bond_index_excess = bond_index[["Q1", "Q2", "Q3", "Q4"]] - money_market[["Q1", "Q2", "Q3","Q4"]]
int_growth_excess = int_growth[["Q1", "Q2", "Q3", "Q4"]] - money_market[["Q1", "Q2", "Q3","Q4"]]
metal_excess = metal[["Q1", "Q2", "Q3", "Q4"]] - money_market[["Q1", "Q2", "Q3","Q4"]]
stock_index_excess = stock_index[["Q1", "Q2", "Q3", "Q4"]] - money_market[["Q1", "Q2", "Q3","Q4"]]
int_value_excess = int_value[["Q1", "Q2", "Q3", "Q4"]] - money_market[["Q1", "Q2", "Q3","Q4"]]
reit_excess = reit[["Q1", "Q2", "Q3", "Q4"]] - money_market[["Q1", "Q2", "Q3","Q4"]]

In [4]:
# a demo
print(bond_index_excess)

        Q1      Q2      Q3      Q4
0  -0.0186 -0.0065 -0.0051  0.0102
1   0.0070  0.0122  0.0044  0.0008
2   0.0298  0.0223  0.0027 -0.0334
3   0.0163 -0.0181  0.0115 -0.0065
4   0.0190  0.0196  0.0016  0.0171
5  -0.0008 -0.0243  0.0052 -0.0020
6   0.0024  0.0213  0.0152  0.0010
7   0.0022  0.0223  0.0398  0.0094
8   0.0168  0.0357  0.0239 -0.0138
9  -0.0001  0.0163  0.0361  0.0003
10  0.0121 -0.0169 -0.0098  0.0372
11  0.0017 -0.0194  0.0162  0.0185


In [5]:
# reshape to make a vector
bond_excess_vec = np.array(bond_index_excess[["Q1", "Q2", "Q3", "Q4"]]).reshape((-1,1))
np.cov(bond_excess_vec.T, bias = True)

array(0.00027117)

In [6]:
# the professor forgets to divide the S.T @ S by the number of observations. If you multiply the above result by 48, you get 

In [7]:
0.00027117*48

0.013016159999999999

# Question 2, Calculate Var-Cov matrix

### Remark 1: The professor uses the excess return data (i.e. return - risk_free) to calculate cov, don't know why
### Remark 2: The professor does not change the data to yearly basis, but simply uses quarterly returns
### Remark 3: The professor forgets to divide the value by the number of observations. So every number is 48 times larger than it should be. But this should not affect later problems as this is linear.

We use the brute-force way to calculate the var-cov matrix.

Step 1: get excess-return data

Step 2: reshape, column stack, to get a matrix

Step 3: demean each column

Step 4: $C = \frac{1}{\text{# of observations}} S S^T$

In [18]:
# a correct demo for calculating the variance of bond_index_excess return

# step 1 and 2, get excess-return data, change it to a column
bond_excess_vec = np.array(bond_index_excess[["Q1", "Q2", "Q3", "Q4"]]).reshape((-1,1))

# step 3, demean the data
bond_excess_vec = bond_excess_vec - np.mean(bond_excess_vec)

# step 4
var_bond = bond_excess_vec.T @ bond_excess_vec / len(bond_excess_vec)
print("Variance of excess bond index return is ", var_bond[0,0])
print("This matches with the answer that the professor sent to me.")

Variance of excess bond index return is  0.0002711709722222222
This matches with the answer that the professor sent to me.


## In order to confirm further computations here we assume that the professor's matrix is correct. That is to say, we do not divide by the number of observations here.

### Below we replicate the professor's "wrong" matrix

In [34]:
# reshape to a vector
bond_excess_vec = np.array(bond_index_excess[["Q1", "Q2", "Q3", "Q4"]]).reshape((-1,1))
int_growth_excess_vec = np.array(int_growth_excess[["Q1","Q2","Q3","Q4"]]).reshape((-1,1))
metal_excess_vec = np.array(metal_excess[["Q1","Q2","Q3","Q4"]]).reshape((-1,1))
stock_excess_vec = np.array(stock_index_excess[["Q1","Q2","Q3","Q4"]]).reshape((-1,1))
int_value_excess_vec = np.array(int_value_excess[["Q1","Q2","Q3","Q4"]]).reshape((-1,1))
reit_excess_vec = np.array(reit_excess[["Q1","Q2","Q3","Q4"]]).reshape((-1,1))

# demean each vector
bond_excess_vec = bond_excess_vec - np.mean(bond_excess_vec)
int_growth_excess_vec = int_growth_excess_vec - np.mean(int_growth_excess_vec)
metal_excess_vec = metal_excess_vec - np.mean(metal_excess_vec)
stock_excess_vec = stock_excess_vec - np.mean(stock_excess_vec)
int_value_excess_vec = int_value_excess_vec - np.mean(int_value_excess_vec)
reit_excess_vec = reit_excess_vec - np.mean(reit_excess_vec)

return_mat = np.hstack([bond_excess_vec, int_growth_excess_vec, metal_excess_vec, stock_excess_vec, \
                        int_value_excess_vec, reit_excess_vec])

# check if indeed has mean zero
print("Close to zero?", np.isclose(np.sum(return_mat, axis = 0), np.zeros(6)))

print("______________________________________________________")
print("this should be the correct var-cov matrix:")
print(pd.DataFrame(return_mat.T @ return_mat/ len(bond_excess_vec)))
print()
print("Compare with the cov matrix calculated from numpy's built in function:")
true_cov_mat = np.cov(return_mat.T, bias= True)
print(pd.DataFrame(true_cov_mat))

print()
print()
print("However, the professor forgot to divide each entry by the number of observations.")
print("Professor's answer:")
prof_cov_mat = return_mat.T @ return_mat
print(pd.DataFrame(prof_cov_mat))

Close to zero? [ True  True  True  True  True  True]
______________________________________________________
this should be the correct var-cov matrix:
          0         1         2         3         4         5
0  0.000271 -0.000165  0.000422 -0.000305 -0.000175  0.000237
1 -0.000165  0.010713  0.010735  0.007441  0.009745  0.007536
2  0.000422  0.010735  0.024614  0.006975  0.009980  0.008250
3 -0.000305  0.007441  0.006975  0.006736  0.007029  0.007345
4 -0.000175  0.009745  0.009980  0.007029  0.009283  0.007224
5  0.000237  0.007536  0.008250  0.007345  0.007224  0.014327

Compare with the cov matrix calculated from numpy's built in function:
          0         1         2         3         4         5
0  0.000271 -0.000165  0.000422 -0.000305 -0.000175  0.000237
1 -0.000165  0.010713  0.010735  0.007441  0.009745  0.007536
2  0.000422  0.010735  0.024614  0.006975  0.009980  0.008250
3 -0.000305  0.007441  0.006975  0.006736  0.007029  0.007345
4 -0.000175  0.009745  0.009980  